In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
#To Read Data from IRIS Dataset
iris = pd.read_csv("https://raw.githubusercontent.com/aiforsec/fds-datasets/main/iris-modified.csv")

#To Display Data along with its Header
iris.head()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
#To get Descriptive Summary of the Dataset
iris.describe()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width
count,100.000000,100.000000,100.000000,100.000000
mean,5.471000,3.094000,2.862000,0.785000
std,0.641698,0.476057,1.448565,0.566288
min,4.300000,2.000000,1.000000,0.100000
25%,5.000000,2.800000,1.500000,0.200000
50%,5.400000,3.050000,2.450000,0.800000
75%,5.900000,3.400000,4.325000,1.300000
max,7.000000,4.400000,5.100000,1.800000


In [ ]:
from sklearn.model_selection import train_test_split

#To add Data to X and y Variables
X = iris.iloc[:, :-1].values
y = iris.iloc[:, -1].values

#To split data for Training and Testing (80% Training and 20% Testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
X_train

80

In [ ]:
y_train

array(['Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa',
       'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iri

In [ ]:
#To get Total No. of Samples and Total No. of Features of Training Dataset
samples = X_train.shape[0]
feature = X_train.shape[1]

In [ ]:
#To get Name of Distinct Classes
classes = np.unique(y_train)

#To get Total No of Classes
number_of_classes = len(classes)
number_of_classes

2

In [ ]:
classes

array(['Iris-setosa', 'Iris-versicolor'], dtype=object)

In [ ]:
# naive bayes trainig model for each class goes here

# create a dictionary to split data by class
# key is name of class
# value is numpy array with shape (no. samples in class, no. features)
def categorize_training_data(measurements, classifications):
  categorized = {}
  classes = np.unique(classifications)
  features = measurements.shape[1]
  for class_value in classes:
    indices = np.where(classifications == class_value)[0] # get every index corresponding to one class
    data = np.zeros((len(indices), features)) # shape is (no. samples in class, no. features)
    for i, index in enumerate(indices):
      data[i] = measurements[index]
    categorized[class_value] = data
  return categorized

# get mean and sd for each feature for each class
# another dictionary
# key is name of class
# value is numpy array of size (features, 2)
# columns are 0=mean, 1=standard deviation
# for iris dataset, rows are 0=sepal_length, 1=sepal_width, 2=petal_length, 3=petal_width
def summarize(sorted_data):
  stats = {}
  for class_value in sorted_data.keys():
    data = sorted_data[class_value].T # just seems like it will be easier if transposed
    summary_stats = np.zeros((data.shape[0],2))
    for i, row in enumerate(data):
      summary_stats[i] = [np.mean(row), np.std(row)]
    stats[class_value] = summary_stats
  return stats

sorted = categorize_training_data(X_train, y_train)
sorted
summarize(sorted)

{'Iris-setosa': array([[5.045     , 0.3323778 ],
        [3.42      , 0.3756328 ],
        [1.4775    , 0.14402691],
        [0.2475    , 0.10951598]]),
 'Iris-versicolor': array([[5.93      , 0.51195703],
        [2.7725    , 0.32092639],
        [4.2525    , 0.47748691],
        [1.33      , 0.20396078]])}

In [ ]:
# calculate probabilities

# get probability of each class
# dictionary: key is class, value is probability
def class_probabilities(classifications):
  class_prob = {}
  classes = np.unique(classifications)
  for class_value in classes:
    class_prob[class_value] = len(np.where(classifications == class_value)[0]) / len(classifications)
  return class_prob

# got this equation from the internet
# lets you calculate the probability of a number (data_point) on a Gaussian distribution
def gaussian_probability(data_point, summary_stats):
  mean = summary_stats[0]
  stdev = summary_stats[1]
  a = math.exp(-((data_point - mean) ** 2 / (2 * stdev ** 2)))
  b = math.sqrt(2 * math.pi) * stdev
  return a / b

# data frame maker to make next function cleaner
def data_frame_maker(data, categories):
  cols = list(np.unique(categories))
  cols.extend(["prediction", "actual"])
  number_rows = data.shape[0]
  number_columns = len(cols)
  array = np.zeros((number_rows, number_columns))
  df = pd.DataFrame(array, columns = cols)
  return df

# predict test data
# data frame
# columns = liklihood of each class, prediction (highest liklihood), actual
# liklihood of each is product of gaussian prob of each feature and class prob
def test_model(x_training, y_training, x_testing, y_testing):
  categorized_training_data = categorize_training_data(x_training, y_training)
  training_data_statistics = summarize(categorized_training_data)
  training_class_probabilities = class_probabilities(y_training)
  num_features = x_training.shape[1]
  num_samples = x_testing.shape[0]
  df = data_frame_maker(x_testing, y_training)
  for i in np.arange(num_samples):
    # for each class
    for class_value in training_class_probabilities.keys():
      # retrieve class probability
      class_prob = training_class_probabilities[class_value]
      # make list of gaussian probabilities
      gaussian_probs = np.zeros(num_features)
      for j in np.arange(num_features):
        gaussian_probs[j] = gaussian_probability(x_testing[i][j], training_data_statistics[class_value][j])
      # turn list into numpy array to find product of array and class prob
      probability = np.prod(gaussian_probs) * class_prob
      # add to data frame (df)
      df.loc[i, class_value] = probability
  # make prediction by determining higher probability -- add to df
  df["prediction"] = df.idxmax(1)
  # add actual class to df to compare
  df["actual"] = y_testing
  return df

In [ ]:
# Initialize an empty dictionary to store all the probabilities
feature_probability = {}
# addressed in a different way

In [ ]:
#Output is the probability of each data entry in the the test dataset (20%). Write code to enumerate it.
# x,sf,we, -> class (versicolor): 0.6

# data under Iris-setosa refers to probability that the sample is Iris-setosa
# data under Iris-versicolor refers to probability that the sample is Iris-versicolor
# data under prediction refers to the species with the higher probability from the first 2 columns
# data under actual refers to what the sample species actually is
test_model(X_train, y_train, X_test, y_test)

,Iris-setosa,Iris-versicolor,prediction,actual
0,1.680954e+00,1.367749e-13,Iris-setosa,Iris-setosa
1,1.441144e-142,6.910880e-02,Iris-versicolor,Iris-versicolor
2,1.346467e+00,1.958438e-17,Iris-setosa,Iris-setosa
3,4.855775e-117,6.165974e-01,Iris-versicolor,Iris-versicolor
4,2.127852e-118,2.350363e-01,Iris-versicolor,Iris-versicolor
5,7.111475e-47,1.899585e-03,Iris-versicolor,Iris-versicolor
6,2.851938e-01,1.473193e-16,Iris-setosa,Iris-setosa
7,7.892780e-128,3.926269e-01,Iris-versicolor,Iris-versicolor
8,3.884250e-135,2.300521e-01,Iris-versicolor,Iris-versicolor
9,4.766041e-95,4.515173e-01,Iris-versicolor,Iris-versicolor
